## Construir un modelo que prediga si una venta significativa de insider generará un movimiento negativo en el precio — es decir, si "sabían algo" antes de que la acción baje.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [3]:
# lectura de datos de tablas fct
ml_dataset = pd.read_csv(('../Dataset/ml_dataset/AAPL_ml_dataset.csv'))
ml_dataset.head()

,Unnamed: 0.1,Unnamed: 0,trade_date,ticker,insider_name,title,trade_type,quantity_of_shares,owned,delta_owned,value,Open,Close,movimiento,significant_transaction,impacto_negativo,año,mes
0,0,0,2015-01-23,AAPL,Riccio Daniel J.,SVP,S,3804,0,100,428955,24.98,25.13,0.15,1,False,2015,1
1,1,1,2015-02-18,AAPL,Jung Andrea,Dir,S,40000,14595,73,5125200,28.50,28.75,0.25,1,False,2015,2
2,2,2,2015-03-06,AAPL,Maestri Luca,CFO,S,3400,14124,19,437920,28.68,28.27,-0.41,1,True,2015,3
3,3,3,2015-03-09,AAPL,Maestri Luca,CFO,S,2800,11324,20,361116,28.58,28.39,-0.19,1,True,2015,3
4,4,4,2015-03-18,AAPL,Maestri Luca,CFO,S,10823,501,96,1394219,28.36,28.69,0.33,1,False,2015,3


In [4]:
ml_dataset = ml_dataset.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

In [5]:
ml_dataset

,trade_date,ticker,insider_name,title,trade_type,quantity_of_shares,owned,delta_owned,value,Open,Close,movimiento,significant_transaction,impacto_negativo,año,mes
0,2015-01-23,AAPL,Riccio Daniel J.,SVP,S,3804,0,100,428955,24.98,25.13,0.15,1,False,2015,1
1,2015-02-18,AAPL,Jung Andrea,Dir,S,40000,14595,73,5125200,28.50,28.75,0.25,1,False,2015,2
2,2015-03-06,AAPL,Maestri Luca,CFO,S,3400,14124,19,437920,28.68,28.27,-0.41,1,True,2015,3
3,2015-03-09,AAPL,Maestri Luca,CFO,S,2800,11324,20,361116,28.58,28.39,-0.19,1,True,2015,3
4,2015-03-18,AAPL,Maestri Luca,CFO,S,10823,501,96,1394219,28.36,28.69,0.33,1,False,2015,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,2024-10-04,AAPL,Maestri Luca,CFO,S,59305,107788,35,13433769,227.40,226.30,-1.10,1,True,2024,10
217,2024-11-15,AAPL,Levinson Arthur D,Dir,S,200000,4215576,5,45464500,226.15,224.75,-1.40,0,False,2024,11
218,2024-11-18,AAPL,Kondo Chris,PAO,S,4130,15419,21,945233,225.00,227.77,2.77,1,False,2024,11
219,2024-12-16,AAPL,Williams Jeffrey E,COO,S,100000,389944,20,24997395,247.72,250.76,3.04,1,False,2024,12


In [6]:
ml_dataset['impacto_negativo'].value_counts()

impacto_negativo
False    171
True      50
Name: count, dtype: int64

In [7]:
50/171

0.29239766081871343

In [8]:
columnas_a_codificar = ['significant_transaction', 'impacto_negativo', 'año', 'mes','title']
final_ml_dataset = pd.get_dummies(ml_dataset[columnas_a_codificar], drop_first=True)

In [9]:
final_ml_dataset

,significant_transaction,impacto_negativo,año,mes,title_CFO,title_COO,title_Dir,title_PAO,title_SVP
0,1,False,2015,1,False,False,False,False,True
1,1,False,2015,2,False,False,True,False,False
2,1,True,2015,3,True,False,False,False,False
3,1,True,2015,3,True,False,False,False,False
4,1,False,2015,3,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...
216,1,True,2024,10,True,False,False,False,False
217,0,False,2024,11,False,False,True,False,False
218,1,False,2024,11,False,False,False,True,False
219,1,False,2024,12,False,True,False,False,False


In [10]:
entrenamiento_validacion, prueba = train_test_split(final_ml_dataset, test_size=0.2, random_state=12345)

entrenamiento, validacion = train_test_split(entrenamiento_validacion, test_size=0.2, random_state=12345)

In [11]:
entrenamiento['impacto_negativo'].value_counts()

impacto_negativo
False    105
True      35
Name: count, dtype: int64

In [12]:
entrenamiento_caracteristicas = entrenamiento.drop(['impacto_negativo'], axis=1)
entrenamiento_objetivo = entrenamiento['impacto_negativo']

validacion_caracteristicas = entrenamiento.drop(['impacto_negativo'], axis=1)
validacion_objetivo = entrenamiento['impacto_negativo']

prueba_caracteristicas = entrenamiento.drop(['impacto_negativo'], axis=1)
prueba_objetivo = entrenamiento['impacto_negativo']

In [13]:
reg_log = LogisticRegression(random_state=12345)

In [14]:
reg_log.fit(entrenamiento_caracteristicas, entrenamiento_objetivo)

LogisticRegression(random_state=12345)

In [15]:
prediccion_entrenamiento = reg_log.predict(entrenamiento_caracteristicas)

In [16]:
pd.Series(prediccion_entrenamiento).value_counts()

False    140
Name: count, dtype: int64

# COMENTARIO
Aquí vemos que el RL no me sirve ya que no me está arrojando datos TRUE que quiere decir que no impactó el precio de la acción. Por lo tanto lo descarto

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
arbol_clf = DecisionTreeClassifier(random_state=12345)

In [19]:
arbol_clf.fit(entrenamiento_caracteristicas, entrenamiento_objetivo)

DecisionTreeClassifier(random_state=12345)

In [20]:
prediccion_arbol = arbol_clf.predict(entrenamiento_caracteristicas)

In [21]:
pd.Series(prediccion_arbol).value_counts()

False    116
True      24
Name: count, dtype: int64

# COMENTARIO
Con Decision Tree se observa que me arroja unos resultados como true y realizo la relación frente a los falsos.

In [23]:
predic_caracteristicas = arbol_clf.predict(validacion_caracteristicas)

In [24]:
predic_caracteristicas.sum()

np.int64(24)

In [25]:
validacion_objetivo.sum()

np.int64(35)

In [26]:
24/35

0.6857142857142857

Realizo el predict ahora con validacion características, luego hago la relación con el total de falsos, dando el 68%


In [248]:
predic_prueba_arbol = arbol_clf.predict(validacion_caracteristicas)

predic_prueba_arbol.sum()

np.int64(24)

In [249]:
prueba_objetivo.sum()

np.int64(35)

In [250]:
24/35

0.6857142857142857

In [251]:
arbol_clf.get_depth()

17

# COMENTARIO
Aquí reduzco las capas del árbol para comprobar si hay cambios en la predicción

In [75]:
arbol_clf_controlado = DecisionTreeClassifier(max_depth=10, random_state=12345)

In [76]:
arbol_clf_controlado.fit(entrenamiento_caracteristicas, entrenamiento_objetivo)

DecisionTreeClassifier(max_depth=10, random_state=12345)

In [77]:
predict_arbol_prueba_controlado = arbol_clf_controlado.predict(prueba_caracteristicas)

In [78]:
sum_predict = predict_arbol_prueba_controlado.sum()
sum_validacion = validacion_objetivo.sum()

print(sum_predict)
print(sum_validacion)
print(sum_predict / sum_validacion)

26
35
0.7428571428571429


In [69]:
final_ml_dataset['impacto_negativo'].value_counts()

impacto_negativo
False    171
True      50
Name: count, dtype: int64

In [85]:
# revisión para decidir si aumento o disminuyo el tamaño de la muestra
50/(171+50)*100


22.624434389140273

# COMENTARIO
Solo el 22% de mis valores verdaderos representan la totalidad de mis registros

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.metrics import classification_report

In [80]:
predict_arbol_valid_controlado = arbol_clf_controlado.predict(validacion_caracteristicas)

In [ ]:
#metricas(arbol_clf_controlado, predict_arbol_prueba_controlado, validacion_objetivo)
print('La recuperacion con datos de validacion es de:', recall_score(validacion_objetivo, predict_arbol_valid_controlado))
print('La precision con datos de validacion es de:', precision_score(validacion_objetivo, predict_arbol_valid_controlado))
print('El f1 con datos de validacion es de:', f1_score(validacion_objetivo, predict_arbol_valid_controlado))

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 26

La recuperacion con datos de validacion es de: 0.6
La precision con datos de validacion es de: 0.8076923076923077
El f1-score con datos de validacion es de: 0.6885245901639344


In [82]:
print(classification_report(validacion_objetivo, predict_arbol_valid_controlado))


              precision    recall  f1-score   support

       False       0.88      0.95      0.91       105
        True       0.81      0.60      0.69        35

    accuracy                           0.86       140
   macro avg       0.84      0.78      0.80       140
weighted avg       0.86      0.86      0.86       140



# COMENTARIO
Con esta evaluación, decido por hacer sobremuestreo


In [90]:
from sklearn.utils import shuffle

In [91]:
def sobremuestreo(caracteristicas, objetivo, tasa_de_crecimiento):

    caracteristicas_0 = caracteristicas[objetivo == 0]
    caracteristicas_1 = caracteristicas[objetivo == 1]

    objetivo_0 = objetivo[objetivo == 0]
    objetivo_1 = objetivo[objetivo == 1]

    caracteristicas_sobre = pd.concat([caracteristicas_0] + [caracteristicas_1] * tasa_de_crecimiento)
    objetivo_sobre = pd.concat([objetivo_0] + [objetivo_1] * tasa_de_crecimiento)

    caracteristicas_sobre, objetivo_sobre = shuffle(
        caracteristicas_sobre,
        objetivo_sobre,
        random_state=12345
    )

    return caracteristicas_sobre, objetivo_sobre

In [95]:
# calculo la tasa de crecimiento
171/50

3.42

In [96]:
entrenamiento_caracteristica_sobre, entrenamiento_objetivo_sobre = sobremuestreo(entrenamiento_caracteristicas, entrenamiento_objetivo, 2)

In [97]:
entrenamiento_objetivo_sobre.value_counts()

impacto_negativo
False    105
True      70
Name: count, dtype: int64

In [102]:
entrenamiento_caracteristica_sobre.shape

(175, 8)

In [104]:
entrenamiento_caracteristicas.shape

(140, 8)

# COMENTARIO
Mi tasa de crecimiento es 2, ya que antes de aplicar sobremuestreo, el grupo de resultados de mi variable objetivo representaba el 22%, ahora representa el 66%. Con esto los datos aumentaron a 35 nuevos registros sintéticos, se puede ver la compración con .shape

No quise aplicar una tasa de crecimiento = 3 ya que igualaría la totalidad de los verdaderos y positivos, según mi punto de vista aumentaría en mayor forma el sesgo. Mayor tasa de crecimiento, mayor sesgo.

In [124]:
def metricas(modelo, pred_valid, real_valid):
    print('El total de registros en validacion es: {}'.format(pred_valid.shape))
    print('El total de impactos negativos reales es: {}'.format(real_valid.sum()))
    print('El total de impactos negativos que se predice es: {}'.format(pred_valid.sum()))
    print()
    print('La recuperacion con datos de validacion es de: {}'.format(recall_score(real_valid, pred_valid)))
    print('La precision con datos de validacion es de: {}'.format(precision_score(real_valid, pred_valid)))
    print('El f1-score con datos de validacion es de: {}'.format(f1_score(real_valid, pred_valid)))
    

In [125]:
metricas(arbol_clf_controlado, predict_arbol_valid_controlado, validacion_objetivo)

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 26

La recuperacion con datos de validacion es de: 0.6
La precision con datos de validacion es de: 0.8076923076923077
El f1-score con datos de validacion es de: 0.6885245901639344


In [109]:
from sklearn.ensemble import RandomForestClassifier

In [234]:
bosque_clf_sobre = RandomForestClassifier(n_estimators=500, random_state=12345)

In [235]:
bosque_clf_sobre.fit(entrenamiento_caracteristica_sobre, entrenamiento_objetivo_sobre)


RandomForestClassifier(n_estimators=500, random_state=12345)

In [236]:
predict_bosque_valid_sobre = bosque_clf_sobre.predict(validacion_caracteristicas)
metricas(bosque_clf_sobre, predict_bosque_valid_sobre, validacion_objetivo)

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 38

La recuperacion con datos de validacion es de: 0.8857142857142857
La precision con datos de validacion es de: 0.8157894736842105
El f1-score con datos de validacion es de: 0.8493150684931506


In [237]:
bosque_clf = RandomForestClassifier(n_estimators=500, random_state=12345)

In [238]:
bosque_clf.fit(entrenamiento_caracteristicas, entrenamiento_objetivo)

RandomForestClassifier(n_estimators=500, random_state=12345)

In [239]:
predict_bosque_valid = bosque_clf.predict(validacion_caracteristicas)
metricas(bosque_clf, predict_bosque_valid, validacion_objetivo)

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 26

La recuperacion con datos de validacion es de: 0.7142857142857143
La precision con datos de validacion es de: 0.9615384615384616
El f1-score con datos de validacion es de: 0.819672131147541


# COMPARACIÓN DE RANDOM FOREST USANDO LOS DATOS ORIGINALES Y DATOS SOBREMUESTREADOS

SOBREMUESTREO

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 38

La recuperacion con datos de validacion es de: 0.8857142857142857
La precision con datos de validacion es de: 0.8157894736842105
El f1-score con datos de validacion es de: 0.8493150684931506

ORIGINAL

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 26

La recuperacion con datos de validacion es de: 0.7142857142857143
La precision con datos de validacion es de: 0.9615384615384616
El f1-score con datos de validacion es de: 0.819672131147541

In [187]:
from sklearn.ensemble import BaggingClassifier

In [198]:
bag_clf = BaggingClassifier(
    estimator=RandomForestClassifier(),
    n_estimators=500,
    max_samples=150,
    random_state=12345

)

In [199]:
bag_clf.fit(entrenamiento_caracteristica_sobre, entrenamiento_objetivo_sobre)

BaggingClassifier(estimator=RandomForestClassifier(), max_samples=150,
                  n_estimators=500, random_state=12345)

In [200]:
pred_bag_clf = bag_clf.predict(validacion_caracteristicas)
metricas(bag_clf, pred_bag_clf, validacion_objetivo)


El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 43

La recuperacion con datos de validacion es de: 0.9142857142857143
La precision con datos de validacion es de: 0.7441860465116279
El f1-score con datos de validacion es de: 0.8205128205128205


In [202]:
bag_clf_arbol = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_features='sqrt', splitter="random"),
    n_estimators=500,
    max_samples=150,
    random_state=12345,
    n_jobs=-1
    
)

bag_clf_arbol.fit(entrenamiento_caracteristica_sobre, entrenamiento_objetivo_sobre)
pred_bag_clf_arbol = bag_clf_arbol.predict(validacion_caracteristicas)
metricas(bag_clf_arbol, pred_bag_clf_arbol, validacion_objetivo)

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 38

La recuperacion con datos de validacion es de: 0.8857142857142857
La precision con datos de validacion es de: 0.8157894736842105
El f1-score con datos de validacion es de: 0.8493150684931506


In [208]:
from sklearn.ensemble import GradientBoostingClassifier

In [276]:
grad_clf = GradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.15,
    max_depth=30,
    random_state=12345,
)

grad_clf.fit(entrenamiento_caracteristica_sobre, entrenamiento_objetivo_sobre)
pred_grad_clf = grad_clf.predict(validacion_caracteristicas)
metricas(grad_clf, pred_grad_clf, validacion_objetivo)

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 38

La recuperacion con datos de validacion es de: 0.8857142857142857
La precision con datos de validacion es de: 0.8157894736842105
El f1-score con datos de validacion es de: 0.8493150684931506


In [247]:
metricas(arbol_clf_controlado, predict_arbol_prueba_controlado, validacion_objetivo)

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 26

La recuperacion con datos de validacion es de: 0.6
La precision con datos de validacion es de: 0.8076923076923077
El f1-score con datos de validacion es de: 0.6885245901639344


# COMENTARIO
A continuación presentaré las métricas realizadas a lo largo del proyecto:


In [262]:
print(f"Las metricas de la validacion son:")
print()
print("ARBOL DE DECISION CONTROLADO")
print()
print(metricas(arbol_clf_controlado, predict_arbol_prueba_controlado, validacion_objetivo))
print()
print("RANDOM FOREST SOBREMUESTREADO")
print(metricas(bosque_clf_sobre, predict_bosque_valid_sobre, validacion_objetivo))
print()
print("RANDOM FOREST CON DATOS ORIGINALES")
print(metricas(bosque_clf, predict_bosque_valid, validacion_objetivo))
print()
print("BAGGING CON RANDOM FOREST")
print(metricas(bag_clf, pred_bag_clf, validacion_objetivo))
print()
print("BAGGING CON ARBOL DE DECISION")
print(metricas(bag_clf_arbol, pred_bag_clf_arbol, validacion_objetivo))
print()
print("BOOSTING DESCENSO GRADIENTE")
print(metricas(bag_clf_arbol, pred_bag_clf_arbol, validacion_objetivo))
print()

Las metricas de la validacion son:

ARBOL DE DECISION CONTROLADO

El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 26

La recuperacion con datos de validacion es de: 0.6
La precision con datos de validacion es de: 0.8076923076923077
El f1-score con datos de validacion es de: 0.6885245901639344
None

RANDOM FOREST SOBREMUESTREADO
El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 38

La recuperacion con datos de validacion es de: 0.8857142857142857
La precision con datos de validacion es de: 0.8157894736842105
El f1-score con datos de validacion es de: 0.8493150684931506
None

RANDOM FOREST CON DATOS ORIGINALES
El total de registros en validacion es: (140,)
El total de impactos negativos reales es: 35
El total de impactos negativos que se predice es: 26

La recuperacion con datos de validacion es de: 0.714


# CONCLUSIONES

* Los modelos Boosting por Descenso de Gradiente, Random Forest con sobremuestreo y Bagging con Árbol de Decisión obtienen los f1-score más altos (alrededor de 0.85), lo que refleja un equilibrio adecuado entre precisión y recall. Estos modelos demuestran un rendimiento consistente y robusto en la detección de impactos negativos.
* En base a los resultados expuestos, personalmente considero que el modelo que mejor ha desplegado las predicciones es el de RANDOM FOREST con la mejor calificación con relación a la métrica f1. Obviamente se puede determinar otro modelo que se acerque con más precisión a las predicciones, sin embargo, como un primer acercamiento a modelos predictivos me pareció interesante dar un repaso por todos los modelos e ir evaluando el impacto de cada uno de ellos.


